In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import re
import fastText as ft


/opt/python/dev/Anaconda2-5.0.1/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
BASE_DIR='/temp0/dev/anjani/'
train_file = BASE_DIR +'us_train_trn02525_10763169889_ascii_train.dat'
train_file_out = BASE_DIR +'us_train_trn02525_10763169889_ascii_train_list.dat'
val_file = BASE_DIR +'us_train_trn02525_10763169889_ascii_val.dat'
test_file = BASE_DIR +'us_train_trn02525_10763169889_ascii_test.dat'
fasttext_pretrain_model ='/temp0/dev/anjani/data/fasttext/tec_item_search/wiki.en.bin'
filename = BASE_DIR+train_file

class_list=[1070238745,1070238723,1070238752,1070240929,1070238731,1070238732,1070238728,1070253159,1070238750,1070238721,1070238749,1070238746,1070238733,1070238736,1070238735,1070238718,1070238716,1070253162,1070238726,1070238725,1070238748,1070238715,1070238739,1070238741,1070238742,1070238730,1070238719,1070238722,1070238729,1070238740,1070238751,1070238720,1070238743,1070238717,1070277868,1070253161,1070238724,1070238727,1070238738,1070238744,1070253160,1070238734,1070238737,1070240898,1070238747]
class_size = len(class_list)

In [10]:
class_dict = dict(zip(class_list,range(len(class_list))))
class_dict_rev = dict(zip(range(len(class_list)),class_list))

In [11]:


ft_w2v_model = ft.load_model(fasttext_pretrain_model)

In [135]:
tf.keras.backend.clear_session()

In [12]:
output_size=len(class_list)
def get_line_w2v(text):
    return np.array(ft_w2v_model.get_sentence_vector(text)).astype('float32')  
def preprocess_text(line):
    line = str.upper(str(line))
    return re.sub('[\[\]\{\}\/,\$\-_:;\'\"&\(\)#]',' ',line)


def _parse_data(line):
    line = tf.py_func(lambda x: preprocess_text(x),[line],tf.string)
    line_split = tf.string_split(source=[line],delimiter="|", skip_empty=False)
    dept_name =  tf.py_func(lambda x: get_line_w2v(x),[line_split.values[1]],tf.float32)
    class_name =  tf.py_func(lambda x: get_line_w2v(x),[line_split.values[2]],tf.float32)
    brand_name =  tf.py_func(lambda x: get_line_w2v(x),[line_split.values[3]],tf.float32)
    desc =  tf.py_func(lambda x: get_line_w2v(x),[line_split.values[4]],tf.float32)
    label = tf.py_func(lambda x: class_dict[int(x)],[line_split.values[5]],tf.int64)
    return tf.reshape(tf.stack([dept_name,class_name,brand_name,desc], axis=0),[-1]),\
tf.one_hot(label,depth=output_size)


def create_features(line):
    line = preprocess_text(line)
    record = line.rstrip().split('|')
    dept_name =get_line_w2v(record[1])
    class_name =get_line_w2v(record[2])
    brand_name =get_line_w2v(record[3])
    desc =get_line_w2v(record[4])
    label = np.eye(class_size)[class_dict[int(record[5])]].astype(np.int64)
    features = np.reshape(np.hstack([dept_name,class_name,brand_name,desc]),1200).astype(np.float32)
    return features,label

def read_csv1(filename):
    with open(filename, 'r') as f:
        for line in f.readlines():
            yield create_features(line)


In [14]:
buffer_size=100
buffer_cnt=0
buffer_str = ""
gen = read_csv1(train_file)
with open(train_file_out, 'w+') as wr:
    for rec in gen:
        buffer_str = buffer_str + repr(list(rec[0]) + list(rec[1])) +"\n"
        buffer_cnt = buffer_cnt + 1
        if buffer_cnt == buffer_size:
            wr.write(buffer_str)
            buffer_cnt = 0
            buffer_str = ""
    wr.write(buffer_str)

IOError: [Errno 28] No space left on device

In [142]:
input_dim=1200
num_dense1 = 300
dropout1 = 0.2
num_dense2 = 300
dropout2 = 0.2
STAMP = 'cnn_dense_%d_%.2f_%d_%.2f'%(num_dense1, dropout1,num_dense2, dropout2)
bst_model_path = BASE_DIR + '/' +STAMP + '.h5'



_input = tf.keras.Input(shape =(input_dim,), name = 'input')


_input_reshape = tf.keras.layers.Reshape((input_dim,1,), name='Reshape')(_input)
cnn_input_layer_1 = tf.keras.layers.Conv1D(filters=5, kernel_size=(3), strides=1, \
                                             padding='valid',activation='relu', name = 'cnn_input_layer_1')(_input_reshape)
pool_input_layer_1 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None, padding='valid',name ='pool_input_layer_1')(cnn_input_layer_1)#

#cnn_input_layer_2 = tf.keras.layers.Conv1D(filters=5, kernel_size=(3), strides=1, \
 #                                            padding='valid',activation='relu', name = 'cnn_input_layer_2')(pool_input_layer_1)
#pool_input_layer_2 = tf.keras.layers.MaxPooling1D(pool_size=2, strides=None, padding='valid',name ='pool_input_layer_2')(cnn_input_layer_2)


x1 = tf.keras.layers.Flatten()(pool_input_layer_1)



_dense_1 = tf.keras.layers.Dense(num_dense1, activation='relu', name = 'dense_layer_1')(x1)
_dense_1 = tf.keras.layers.Dropout(dropout1)(_dense_1)

#_dense_2 = tf.keras.layers.Dense(num_dense2, activation='relu', name = 'dense_layer_2')(_dense_1)
#_dense_2 = tf.keras.layers.Dropout(dropout2)(_dense_2)

_output = tf.keras.layers.Dense(output_size, activation='softmax', name='softmax')(_dense_1)


In [143]:
model = tf.keras.models.Model(inputs=[_input], outputs=[_output])
model.compile(loss='categorical_crossentropy',
        optimizer='nadam',
        metrics=['acc'])

In [144]:
early_stopping =tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6)

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=False)
ReduceLROnPlateau= tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

tbCallBack = tf.keras.callbacks.TensorBoard(log_dir=BASE_DIR, histogram_freq=1, write_graph=True, \
                                            write_images=True,write_grads=True)

In [145]:

d_train_file = tf.data.TextLineDataset(train_file).map(_parse_data)
d_val_file = tf.data.TextLineDataset(val_file).map(_parse_data)
d_test_file = tf.data.TextLineDataset(test_file).map(_parse_data)

batched_train_file = d_train_file.batch(512)
batched_val_file = d_val_file.batch(512)
batched_test_file = d_test_file.batch(512)

train_iterator = batched_train_file.make_one_shot_iterator()
val_iterator = batched_val_file.make_one_shot_iterator()
test_iterator = batched_test_file.make_one_shot_iterator()

In [146]:
hist = model.fit(train_iterator, epochs=100, steps_per_epoch=10000,validation_data=val_iterator, validation_steps=100,  callbacks=[early_stopping, model_checkpoint,ReduceLROnPlateau])

Epoch 1/100
10000/10000 [==============================] - 2926s 293ms/step - loss: 0.2278 - acc: 0.9439 - val_loss: 0.1281 - val_acc: 0.9672
Epoch 2/100
10000/10000 [==============================] - 2934s 293ms/step - loss: 0.1385 - acc: 0.9650 - val_loss: 0.1145 - val_acc: 0.9715
Epoch 3/100
10000/10000 [==============================] - 2932s 293ms/step - loss: 0.1264 - acc: 0.9679 - val_loss: 0.1066 - val_acc: 0.9717
Epoch 4/100
10000/10000 [==============================] - 2945s 294ms/step - loss: 0.1196 - acc: 0.9693 - val_loss: 0.1002 - val_acc: 0.9739
Epoch 5/100
10000/10000 [==============================] - 2930s 293ms/step - loss: 0.1148 - acc: 0.9705 - val_loss: 0.0991 - val_acc: 0.9744
Epoch 6/100
10000/10000 [==============================] - 2914s 291ms/step - loss: 0.1113 - acc: 0.9713 - val_loss: 0.0935 - val_acc: 0.9753
Epoch 7/100
10000/10000 [==============================] - 2928s 293ms/step - loss: 0.1085 - acc: 0.9718 - val_loss: 0.0969 - val_acc: 0.9750
Epoch 

 5313/10000 [==============>...............] - ETA: 22:14 - loss: 0.0871 - acc: 0.9770 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00Epoch 21/100


    0/10000 [..............................] - ETA: 0s - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 00021: ReduceLROnPlateau reducing learning rate to 2.00000013137e-06.


In [147]:
model.evaluate(test_iterator, steps=300)

300/300 [==============================] - 72s 241ms/step


[0.07534120067954063, 0.9797395833333333]

In [148]:
tf.__version__

'1.10.0'